In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
output = [11 + i*6 for i in range(7)]

In [3]:
def loadimage(file):
    matrix = np.genfromtxt(file).T
    return matrix

def normimage(matrix):
    min_ = matrix.min()
    matrix = (matrix - min_) / (matrix.max() - min_)
    return abs(1 - matrix)

In [4]:
w_images = np.array([loadimage("Tungsteno/%dkV_1.txt"%i) for i in output])
ff_w = np.array([loadimage("Tungsteno/%dkV_2.txt"%i) for i in output])
au_images = np.array([loadimage("Plata/%dkV_1.txt"%i) for i in output])
ff_au = np.array([loadimage("Plata/%dkV_2.txt"%i) for i in output])

In [5]:
au_images[-1] = (au_images[-1] + au_images[-2]) / 2

In [6]:
norm_w = w_images / (ff_w + 1)
norm_au = au_images / (ff_au + 1)

for i in range(len(output)):
    norm_w[i] = normimage(norm_w[i])
    norm_au[i] = normimage(norm_au[i])

In [7]:
def animate(i):
    w_im.set_array(norm_w[i])
    au_im.set_array(norm_au[i])
    text.set_text("%s kV"%output[i])
    return w_im, au_im, text

In [8]:
cmap = "jet"

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (8, 4.5))
text = ax1.text(200, 225, "", color = "w")

w_im = ax1.imshow(norm_w[0], vmin = 0, vmax = 1, cmap = cmap)
au_im = ax2.imshow(norm_au[0], vmin = 0, vmax = 1, cmap = cmap)

ax1.set_title("Tungsteno")
ax2.set_title("Plata")
ax1.set_axis_off()
ax2.set_axis_off()

ani = FuncAnimation(fig, animate, frames = len(output), interval = 500)
# ani.save("results.gif", writer = "imagemagick")
plt.show()

In [9]:
from skimage import measure

r = norm_w[-1]

contours = measure.find_contours(r, 0.8)

fig, ax = plt.subplots()

ax.imshow(r, interpolation='nearest', cmap = "jet")

for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2)

plt.show()

In [10]:
def crop(matrix, a, b, c, d):
    X = np.arange(0, 256)
    Y = np.arange(0, 256)

    X, Y = np.meshgrid(X, Y)
    
    pos = (X >= a) & (X <= b) & (Y >= c) & (Y <= d)
    
    T = matrix[pos]
    return T.reshape((d-c + 1), (b-a + 1))

In [11]:
def animate(i):
    e = output[i]
    im = data[i]
    small = crop(im, a, b, c, d)
    p1.set_array(im)
    p2.set_array(small)
    
    M, m = small.max(), small.min()
    contrast = 100*(M - m)
    
    axes[0].set_title("%d kV"%e)
    axes[1].set_title("Contraste: %.2f"%contrast)
    
    return p1, p2

In [12]:
fig, axes = plt.subplots(1, 2)

a, b, c, d = 65, 90, 50, 65

p1 = axes[0].imshow(np.zeros((256, 256)), vmin = 0, vmax = 1, cmap = "jet")
p2 = axes[1].imshow(np.zeros((b - a + 1, d - c + 1)), vmin = 0, vmax = 1, cmap = "jet")

data = norm_w

ani = FuncAnimation(fig, animate, frames = len(output), interval = 500)
ani.save("CW.gif", writer = "imagemagick")
plt.show()

In [13]:
fig, axes = plt.subplots(1, 2)

a, b, c, d = 70, 95, 55, 75

p1 = axes[0].imshow(np.zeros((256, 256)), vmin = 0, vmax = 1, cmap = "jet")
p2 = axes[1].imshow(np.zeros((b - a + 1, d - c + 1)), vmin = 0, vmax = 1, cmap = "jet")

data = norm_au

ani = FuncAnimation(fig, animate, frames = len(output), interval = 500)
ani.save("CAu.gif", writer = "imagemagick")
plt.show()